In [6]:
import os
os.chdir('../')

import transtab

# set random seed
transtab.random_seed(42)

In [7]:
# load a dataset and start vanilla supervised training
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols = transtab.load_data(['credit-g', 'credit-approval'])

# build transtab classifier model
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)

# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'val_loss',
    'eval_less_is_better':True,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':1e-4,
    'weight_decay':1e-4,
    }
transtab.train(model, trainset[0], valset[0], **training_arguments)

# save model
model.save('./ckpt/pretrained')

########################################
openml data index: 31
load data from credit-g
# data: 1000, # feat: 20, # cate: 11,  # bin: 2, # numerical: 7, pos rate: 0.70
########################################
openml data index: 29
load data from credit-approval
# data: 690, # feat: 15, # cate: 9,  # bin: 0, # numerical: 6, pos rate: 0.56


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0, test val_loss: 0.574102
epoch: 0, train loss: 3.9759, lr: 0.000100, spent: 0.4 secs
epoch: 1, test val_loss: 0.565162
epoch: 1, train loss: 3.7812, lr: 0.000100, spent: 0.9 secs
epoch: 2, test val_loss: 0.576745
EarlyStopping counter: 1 out of 5
epoch: 2, train loss: 3.6560, lr: 0.000100, spent: 1.1 secs
epoch: 3, test val_loss: 0.566665
EarlyStopping counter: 2 out of 5
epoch: 3, train loss: 3.6539, lr: 0.000100, spent: 1.4 secs
epoch: 4, test val_loss: 0.548929
epoch: 4, train loss: 3.6118, lr: 0.000100, spent: 1.7 secs
epoch: 5, test val_loss: 0.545800
epoch: 5, train loss: 3.5634, lr: 0.000100, spent: 2.2 secs
epoch: 6, test val_loss: 0.545121
epoch: 6, train loss: 3.5035, lr: 0.000100, spent: 2.4 secs
epoch: 7, test val_loss: 0.529130
epoch: 7, train loss: 3.4372, lr: 0.000100, spent: 2.7 secs
epoch: 8, test val_loss: 0.525149
epoch: 8, train loss: 3.3768, lr: 0.000100, spent: 3.0 secs
epoch: 9, test val_loss: 0.518042
epoch: 9, train loss: 3.3204, lr: 0.000100, spent: 3

2022-10-05 08:35:04.023 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2022-10-05 08:35:04.042 | INFO     | transtab.trainer:save_model:243 - saving model checkpoint to ./checkpoint
2022-10-05 08:35:04.167 | INFO     | transtab.trainer:train:141 - training complete, cost 13.1 secs.


epoch: 38, test val_loss: 0.503903
EarlyStopping counter: 5 out of 5
early stopped


In [8]:
# now let's use another data and try to leverage the pretrained model for finetuning
# here we have loaded the required data `credit-approval` before, no need to load again.

# load the pretrained model
model.load('./ckpt/pretrained')

# update model's categorical/numerical/binary column dict
# need to specify the number of classes if the new dataset has different # of classes from the 
# pretrained one.
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols, 'num_class':2})

2022-10-05 08:35:04.352 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-10-05 08:35:04.354 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-10-05 08:35:04.355 | INFO     | transtab.modeling_transtab:load:775 - load model from ./ckpt/pretrained
2022-10-05 08:35:04.370 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from ./ckpt/pretrained/extractor/extractor.json
2022-10-05 08:35:04.372 | INFO     | transtab.modeling_transtab:update:832 - Build a new classifier with num 2 classes outputs, need further finetune to work.


In [9]:
# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':2e-4,
    }

transtab.train(model, trainset[1], valset[1], **training_arguments)


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)
/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.282251
epoch: 0, train loss: 3.3862, lr: 0.000200, spent: 0.2 secs
epoch: 1, test auc: 0.865801
epoch: 1, train loss: 2.8794, lr: 0.000200, spent: 0.3 secs


/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)
/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)


epoch: 2, test auc: 0.865801
epoch: 2, train loss: 2.5943, lr: 0.000200, spent: 0.7 secs
epoch: 3, test auc: 0.865801
epoch: 3, train loss: 2.4300, lr: 0.000200, spent: 0.8 secs


...

epoch: 36, test auc: 0.934199
EarlyStopping counter: 1 out of 5
epoch: 36, train loss: 1.4738, lr: 0.000200, spent: 6.1 secs
epoch: 37, test auc: 0.934199
EarlyStopping counter: 2 out of 5
epoch: 37, train loss: 1.4667, lr: 0.000200, spent: 6.2 secs


/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)
/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)


epoch: 38, test auc: 0.933333
EarlyStopping counter: 3 out of 5
epoch: 38, train loss: 1.4209, lr: 0.000200, spent: 6.3 secs
epoch: 39, test auc: 0.933333
EarlyStopping counter: 4 out of 5
epoch: 39, train loss: 1.4371, lr: 0.000200, spent: 6.4 secs


/home/zifengw2/outcome_predict/transtab/transtab/trainer.py:169: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  y_test = pd.concat(y_test, 0)
2022-10-05 08:35:10.982 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2022-10-05 08:35:10.994 | INFO     | transtab.trainer:save_model:243 - saving model checkpoint to ./checkpoint
2022-10-05 08:35:11.142 | INFO     | transtab.trainer:train:141 - training complete, cost 6.7 secs.


epoch: 40, test auc: 0.929870
EarlyStopping counter: 5 out of 5
early stopped


In [10]:
# evaluation
x_test, y_test = testset[1]
ypred = transtab.predict(model, x_test)
transtab.evaluate(ypred, y_test, metric='auc')

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, ypred))

auc 0.95 mean/interval 0.8757(0.05)
0.8807749627421758
